In [ ]:
# Update Bioconductor

if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.17")

In [ ]:
# Install packages

# ComplexHeatmap
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("ComplexHeatmap")

In [4]:
# Load packages

library(ComplexHeatmap)

In [5]:
# Load in dataframe

df <- read.csv("tables_and_figures/expected_vs_observed_peaks_master.csv", header = TRUE)

[1] "exp_vs_obs.py"          "generate_heatmap.ipynb" "project_files"         
[4] "seq_data"               "snakefiles"             "tables_and_figures"